# Decision Tree Program

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.tree import DecisionTreeClassifier
import sys

#### Read File

In [2]:
def read_file(file, headers, delimiter=','):
    """Function to read a CSV file and will result in error if one is not
       submitted.
       File needs to have the last column as the class.
       Function will take all the columns up to the last column as the 
       features.
   
       2nd argument is a string of headers. Defaults to None if no headers are 
       passed through.
   
       Returns a data frame with the features and a data frame with the class 
       labels.
    """ 

    try:        
        headers = headers.split(',')
        df = pd.read_csv(file, names=headers, delimiter=delimiter)
    except:
        try:
            df = pd.read_csv(file, header=None, delimiter=delimiter)
        except:
            print('Needs a csv file.')
    
   
    features_df = pd.DataFrame(df.iloc[:, 0:-1])
    labels_df = pd.DataFrame(df.iloc[:,-1])
    
    return features_df, labels_df

#### Entropy

In [3]:
def get_entropy(tree_values):
    """Function to get the individual entropies for each branch.    
        Takes the decision tree classifier values method.       
        Returns a list of entropies and list of values for each branch.
    """
    
    # value of each class per branch
    node_splits = tree_values
    
    entropies = []
    values = []
    
    # separate nodes to list of individual arrays for each node in tree
    for node in node_splits:
        entropy = 0
        values.append(node[0])

        for counts in node[0]:
            # calculate probability with unique counts of each node / 
            # total counts
            probabilities = counts / node.sum()
            if probabilities !=0:
                # entropy equation
                entropy += probabilities * -np.log2(probabilities)
        # list of all entropies
        entropies.append(entropy)
    
    return entropies, values

#### Information Gain

In [5]:
def calculate_info_gain(entropies, values, nodes):
    """Function to calculate the information gain per branch.
       Takes the list of entropies, list of values, and nodes per class.
       Returns the information gain.
    """
    
    # calculate weights = sum of values
    n = values[nodes[0]].sum()
    
    # probability of left and right splits over weights
    prob_left = values[nodes[1]].sum()/n
    prob_right = values[nodes[2]].sum()/n
    
    # weighting the entropy of both left and right branches
    overall_entropy = prob_left*entropies[nodes[1]] + \
    prob_right*entropies[nodes[2]]
    
    # current entropy - overall entropy 
    info_gain = entropies[nodes[0]] - overall_entropy
    
    return info_gain

#### Construct The Tree

In [6]:
def build_tree(min_node, min_impurity, data, labels):
    """Function to build the tree with the DecisionTreeClassifier.
       Takes the minimum size of leaf to stop the tree growth,
          minimum impurity (oposite of max impurity), data, and the labels.
       Returns a fitted decision tree.
    """
    
    # build the tree
    dt = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 
                                min_node, min_impurity_decrease = min_impurity)
    
    # fit the tree
    dt.fit(data, labels)
    
    return dt

#### Construct Nodes

In [7]:
def construct_nodes(dt):
    """Function to determine if the node is a decision node or a leaf.
       Takes the decision tree from the decision tree classifier.
       Retuns the node depth and if it is a leaf.
    """
    
    # get information from the modules in the decision tree classifier
    n_nodes = dt.tree_.node_count
    children_left = dt.tree_.children_left
    children_right = dt.tree_.children_right
    feature = dt.tree_.feature
    threshold = dt.tree_.threshold
      
    # The tree structure can be traversed to compute various properties such
    # as the depth of each node and whether or not it is a leaf.
    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth

    # decide if a leaf or a decision
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

        # If we have a decision node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True
    
    
    return node_depth, is_leaves

#### Main

In [8]:
def Decision_Tree(D, min_node, min_impurity, headers=None, delimiter=','):
    """Function to make a decision tree based on entropy and information gain, 
         until max depth is reached or purity.
       D: CSV file
       min_node: minimum size of leaf to stop
       min_impurity: Threshold for early stopping in tree growth. A node will 
          split if it's impurity is above the threshold, otherwise it is a 
          leaf. This is oposite of max impurity.
       headers = Defaults to none, option to put in headers for file.
       delimiter = Delimiter for file reading, defaults to comma.
       Returns a decision tree with decision, leaf, purity, gain, size, and 
          class.
    """
    
    
    try:
        X,y = read_file(D, headers, delimiter)
               
        # build tree
        dt = build_tree(min_node, min_impurity, X, y)

        # get entropies
        entropies, values = get_entropy(dt.tree_.value)

        # determine type
        node_depth, is_leaves = construct_nodes(dt)

        # get info from classifier module
        n_nodes = dt.tree_.node_count
        feature = dt.tree_.feature
        threshold = dt.tree_.threshold

        # create list of classes for each node
        node_labels = []
        for node in dt.tree_.value:
            node_labels.append(dt.classes_[np.argmax(node)])


        # ---Print Outcome---

        for i in range(n_nodes):
            # if it is a leaf
            if is_leaves[i]:            
                depth = node_depth[i]
                class_label = node_labels[i]
                purity = np.round((1-dt.tree_.impurity[i]),2)
                size = dt.tree_.n_node_samples[i]

                print('{}Leaf: Label = {} Purity = {} Size = {}'.format
                      (depth*'\t', class_label, purity, size))

            # if it is a decision
            else: 
                # parent and two children
                nodes = [i, i+1, i+2]

                # calcualte information gain
                info_gain = np.round(calculate_info_gain(entropies, values, 
                                                         nodes),2)
                depth = node_depth[i]
                decision_threshold = np.round(threshold[i],2)

                print('{}Decision: {} <= {} Gain = {}'.format
                      (depth*'\t', X.columns[feature[i]], decision_threshold, 
                       info_gain))
                
    except:
        print('Unexpected error:', sys.exc_info()[0])
        raise

#### Run Program

In [9]:
# minimum leaf size to stop = 5
# purity to stop = .95

Decision_Tree('iris.txt', 5, .05, 
              'sepal_length, sepal_width, petal_length, petal_width, class')

Decision:  petal_length <= 2.45 Gain = 0.92
	Leaf: Label = Iris-setosa Purity = 1.0 Size = 50
	Decision:  petal_width <= 1.75 Gain = 0.69
		Decision:  petal_length <= 4.95 Gain = 0.21
			Leaf: Label = Iris-versicolor Purity = 0.85 Size = 48
			Leaf: Label = Iris-virginica Purity = 0.08 Size = 6
		Leaf: Label = Iris-virginica Purity = 0.85 Size = 46


Resources: 

https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py

https://www.youtube.com/watch?v=Twow3toMQiw&list=PLPOTBrypY74xS3WD0G_uzqPjCQfU6IRK-&index=3&t=0s